# Training a basic SAE with SAELens

This tutorial demonstrates training a simple, relatively small Sparse Autoencoder, specifically on the tiny-stories-1L-21M model.

As the SAELens library is under active development, please open an issue if this tutorial is stale [here](https://github.com/decoderesearch/SAELens).


In [1]:
import torch
import os

from sae_lens import (
    LanguageModelSAERunnerConfig,
    SAETrainingRunner,
    StandardTrainingSAEConfig,
    LoggingConfig,
)

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

/ssd/jdh/interpretability/experiments/exp-sae-lens/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


# Model Selection and Evaluation (Feel Free to Skip)

We'll use the runner to train an SAE on a TinyStories Model. This is a very small model so we can train an SAE on it quite quickly. Before we get started, let's load in the model with `transformer_lens` and see what it can do.

TransformerLens gives us 2 functions that are useful here (and circuits viz provides a third):

1. `transformer_lens.utils.test_prompt` will help us see when the model can infer one token.
2. `HookedTransformer.generate` will help us see what happens when we sample from the model.
3. `circuitsvis.logits.token_log_probs` will help us visualize the log probs of tokens at several positions in a prompt.


In [2]:
from transformer_lens import HookedTransformer

model = HookedTransformer.from_pretrained(
    "tiny-stories-1L-21M"
)  # This will wrap huggingface models and has lots of nice utilities.

`torch_dtype` is deprecated! Use `dtype` instead!


Loaded pretrained model tiny-stories-1L-21M into HookedTransformer


### Getting a vibe for a model using `model.generate`

Let's start by generating some stories using the model.

In [3]:
# here we use generate to get 10 completeions with temperature 1. Feel free to play with the prompt to make it more interesting.
for i in range(5):
    display(
        model.generate(
            "Once upon a time",
            stop_at_eos=False,  # avoids a bug on MPS
            temperature=1,
            verbose=False,
            max_new_tokens=50,
        )
    )

"Once upon a time, there was a bright little triangle. Each day it would glow until it burned in the darkness. The triangle was a bit sad, but it didn't. The picture on the wall had started to rot.\n\nThe triangle stepped closer and closer"

"Once upon a time, there was an old man who didn't know what to do. He stumbled upon everything, but he did as he was asked, he was patient. The man smiled and said he would no longer need to delay his lecture, he had come too"

'Once upon a time, there was a lucky dog named Spot. He loved to jog around in the sun and explore his neighborhood. Today, Spot wanted to find something special for a jog.\n\nHe went to explore the neighborhood, making new jolly sounds that fit'

'Once upon a time, there lived a little girl in a beautiful big desert. She was filled with plants and plenty of water flowing. The farmer was happy to help and she stayed there until the oasis was set in. \n\nOne day, the little girl'

'Once upon a time, there was a child called Jo. Jojo loved and she had a special toy car that she valued very much. She asked her parents, "Can I play with it every day?" \n\nOne day, Jojo was practicing his toy'

One thing we notice is that the model seems to be able to repeat the name of the main character very consistently. It can output a pronoun intead but in some stories will repeat the protagonists name. This seems like an interesting capability to analyse with SAEs. To better understand the models ability to remember the protagonists name, let's extract a prompt where the next character is determined and use the "test_prompt" utility from TransformerLens to check the ranking of the token for that name.

In [4]:
from transformer_lens.utils import test_prompt

# Test the model with a prompt
test_prompt(
    "Once upon a time, there was a little girl named Lily. She lived in a big, happy little town. On her big adventure,",
    " Lily",
    model,
    prepend_space_to_answer=False,
)

Tokenized prompt: ['<|endoftext|>', 'Once', ' upon', ' a', ' time', ',', ' there', ' was', ' a', ' little', ' girl', ' named', ' Lily', '.', ' She', ' lived', ' in', ' a', ' big', ',', ' happy', ' little', ' town', '.', ' On', ' her', ' big', ' adventure', ',']
Tokenized answer: [' Lily']


Performance on answer token:
Rank: 1        Logit: 18.82 Prob:  8.62% Token: | Lily|

Top 0th token. Logit: 21.00 Prob: 76.18% Token: | she|
Top 1th token. Logit: 18.82 Prob:  8.62% Token: | Lily|
Top 2th token. Logit: 18.16 Prob:  4.45% Token: | there|
Top 3th token. Logit: 17.00 Prob:  1.39% Token: | the|
Top 4th token. Logit: 16.76 Prob:  1.10% Token: | her|
Top 5th token. Logit: 16.61 Prob:  0.94% Token: | all|
Top 6th token. Logit: 16.56 Prob:  0.90% Token: | everyone|
Top 7th token. Logit: 16.04 Prob:  0.53% Token: | things|
Top 8th token. Logit: 16.04 Prob:  0.53% Token: | they|
Top 9th token. Logit: 16.03 Prob:  0.53% Token: | people|


Ranks of the answer tokens: [(' Lily', 1)]

In [5]:
import circuitsvis as cv  # optional dep, install with pip install circuitsvis

# Let's make a longer prompt and see the log probabilities of the tokens
example_prompt = """Hi, how are you doing this? I'm really enjoying your posts"""
logits, cache = model.run_with_cache(example_prompt)
cv.logits.token_log_probs(
    model.to_tokens(example_prompt),
    model(example_prompt)[0].log_softmax(dim=-1),
    model.to_string,
)
# hover on the output to see the result.

Let's combine `model.generate` and the token log probs visualization to see the log probs on text generated by the model. Note that we can play with the temperature and this should sample less likely trajectories according to the model. I've increased the maximum number of tokens in order to get a full story.

Some things to explore:

- Which tokens does the model assign high probability to? Can you see how the model should know which word comes next?
- What happens if you increase / decrease the temperature?
- Do the rankings of tokens seem sensible to you? What about where the model doesn't assign a high probability to the token which came next?


In [8]:
example_prompt = model.generate(
    "Once upon a time",
    stop_at_eos=False,  # avoids a bug on MPS
    temperature=1,
    verbose=True,
    max_new_tokens=200,
)
logits, cache = model.run_with_cache(example_prompt)
cv.logits.token_log_probs(
    model.to_tokens(example_prompt),
    model(example_prompt)[0].log_softmax(dim=-1),
    model.to_string,
)

100%|██████████| 200/200 [00:00<00:00, 462.48it/s]


# Training an SAE

Now we're ready to train out SAE. We'll make a runner config, instantiate the runner and the rest is taken care of for us!

During training, you use weights and biases to check key metrics which indicate how well we are able to optimize the variables we care about.

To get a better sense of which variables to look at, you can read my (Joseph's) post [here](https://www.lesswrong.com/posts/f9EgfLSurAiqRJySD/open-source-sparse-autoencoders-for-all-residual-stream) and especially look at my weights and biases report [here](https://links-cdn.wandb.ai/wandb-public-images/links/jbloom/uue9i416.html).

A few tips:

- Feel free to reorganize your wandb dashboard to put L0, CE_Loss_score, explained variance and other key metrics in one section at the top.
- Make a [run comparer](https://docs.wandb.ai/guides/app/features/panels/run-comparer) when tuning hyperparameters.
- You can download the resulting sparse autoencoder / sparsity estimate from wandb and upload them to huggingface if you want to share your SAE with other.
  - cfg.json (training config)
  - sae_weight.safetensors (model weights)
  - sparsity.safetensors (sparsity estimate)


## MLP Out

I've tuned the hyperparameters below for a decent SAE which achieves 86% CE Loss recovered and an L0 of ~85, and runs in about 2 hours on an M3 Max. You can get an SAE that looks better faster if you only consider L0 and CE loss but it will likely have more dense features and more dead features. Here's a link to my output with two runs with two different L1's: https://wandb.ai/jbloom/sae_lens_tutorial .


In [10]:
total_training_steps = 30_000  # probably we should do more
batch_size = 4096
total_training_tokens = total_training_steps * batch_size

lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5  # 20% of training
l1_warm_up_steps = total_training_steps // 20  # 5% of training

cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distibuion)
    model_name="tiny-stories-1L-21M",  # our model (more options here: https://neelnanda-io.github.io/TransformerLens/generated/model_properties_table.html)
    hook_name="blocks.0.hook_mlp_out",  # A valid hook point (see more details here: https://neelnanda-io.github.io/TransformerLens/generated/demos/Main_Demo.html#Hook-Points)
    dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",  # this is a tokenized language dataset on Huggingface for the Tiny Stories corpus.
    is_dataset_tokenized=True,
    streaming=True,  # we could pre-download the token dataset if it was small.
    # SAE Parameters
    sae=StandardTrainingSAEConfig(
        d_in=1024,  # the width of the mlp output.
        d_sae=16384,  # the width of the SAE. Larger will result in better stats but slower training.
        apply_b_dec_to_input=False,  # We won't apply the decoder weights to the input.
        normalize_activations="expected_average_only_in",
        l1_coefficient=5,  # will control how sparse the feature activations are
        l1_warm_up_steps=l1_warm_up_steps,  # this can help avoid too many dead features initially.
    ),
    # Training Parameters
    lr=5e-5,  # lower the better, we'll go fairly high to speed up the tutorial.
    adam_beta1=0.9,  # adam params (default, but once upon a time we experimented with these.)
    adam_beta2=0.999,
    lr_scheduler_name="constant",  # constant learning rate with warmup. Could be better schedules out there.
    lr_warm_up_steps=lr_warm_up_steps,  # this can help avoid too many dead features initially.
    lr_decay_steps=lr_decay_steps,  # this will help us avoid overfitting.
    train_batch_size_tokens=batch_size,
    context_size=512,  # will control the lenght of the prompts we feed to the model. Larger is better but slower. so for the tutorial we'll use a short one.
    # Activation Store Parameters
    n_batches_in_buffer=64,  # controls how many activations we store / shuffle.
    training_tokens=total_training_tokens,  # 100 million tokens is quite a few, but we want to see good stats. Get a coffee, come back.
    store_batch_size_prompts=16,
    # Resampling protocol
    feature_sampling_window=1000,  # this controls our reporting of feature sparsity stats
    dead_feature_window=1000,  # would effect resampling or ghost grads if we were using it.
    dead_feature_threshold=1e-4,  # would effect resampling or ghost grads if we were using it.
    # WANDB
    logger=LoggingConfig(
        log_to_wandb=True,  # always use wandb unless you are just testing code.
        wandb_project="sae_lens_tutorial",
        wandb_log_frequency=30,
        eval_every_n_wandb_logs=20,
    ),
    # Misc
    device=device,
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    dtype="float32",
)
# look at the next cell to see some instruction for what to do while this is running.
sparse_autoencoder = SAETrainingRunner(cfg).run()

/tmp/ipykernel_3042948/1419655237.py:57: DeprecationWarning: Use LanguageModelSAETrainingRunner instead
  sparse_autoencoder = SAETrainingRunner(cfg).run()


Loaded pretrained model tiny-stories-1L-21M into HookedTransformer


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true&ref=models
wandb: Paste an API key from your profile and hit enter:wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /ssd/jdh/.netrc
wandb: Currently logged in as: lasip21784 (lasip21784-freelancer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Training SAE:   0%|          | 0/122880000 [00:00<?, ?it/s]









































































































































































Estimating norm scaling factor: 100%|██████████| 1000/1000 [00:18<00:00, 54.34it/s]
30000| mse_loss: 198.75142 | l1_loss: 160.39059: 100%|██████████| 122880000/122880000 [22:51<00:00, 89570.77it/s]


details/current_learning_rate,████████████████████████████████████▇▅▄▁
details/l1_coefficient,▁▄██████████████████████████████████████
details/n_training_samples,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
losses/l1_loss,▁▆███▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
losses/mse_loss,▆█▇▇▆▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
losses/overall_loss,▄▅▆█▅▅▅▄▄▄▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
metrics/explained_variance,█▃▂▁▁▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▅▄▅▄▅▄▅▅▅▅▅▅▅▅▅▅
metrics/explained_variance_legacy,▅▃▁▁▁▂▃▃▃▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇████▇▇█████████
metrics/explained_variance_legacy_std,▄▁▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂█▂▂▂▂▂▂▂▂▂▂▂▂▂▂
metrics/l0,█▅▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
+5,...
